In [1]:
using Pkg
Pkg.add("SQLite")
Pkg.add("DBInterface")
Pkg.add("Tables")
Pkg.add("DataStructures")
Pkg.add("DataFrames")

   Updating registry at `~/.julia/registries/General`


   Updating 

git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...


   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...


   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...


   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [2]:
using SQLite
using DBInterface
using Tables
using DataStructures
using DataFrames

In [3]:
HEADER = "HL2DEMO\x00"
TYPE_INTEGER_LEN = 4
TYPE_FLOAT_LEN = 4
TYPE_STRING_LEN = 260

260

In [4]:
db = SQLite.DB("testDB.db")

#DBInterface.execute(db, "DROP TABLE IF EXISTS HEADERS")

DBInterface.execute(
    db,
    "CREATE TABLE IF NOT EXISTS HEADERS(
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    DEMONAME TEXT NOT NULL,
    DEMOPROTOCOL INT NOT NULL,
    NETWORKPROTOCOL INT NOT NULL,
    SERVERNAME TEXT NOT NULL,
    CLIENTNAME TEXT NOT NULL,
    MAPNAME TEXT NOT NULL,
    GAMEDIRECTORY TEXT NOT NULL,
    PLAYBACKTIME REAL NOT NULL,
    TICKS INT NOT NULL,
    FRAMES INT NOT NULL,
    SIGNONLENGTH INT NOT NULL,
    TICKRATE REAL NOT NULL,
    STEAMID INT NOT NULL,
    UNIQUE (DEMONAME)
);",
)

SQLite.Query(SQLite.Stmt(SQLite.DB("testDB.db"), 1), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol,Int64}(), Base.RefValue{Int64}(0))

In [5]:
function __read_float(byte_arr, num_bytes = TYPE_FLOAT_LEN)
    buffer = reinterpret(Float32, byte_arr[1:num_bytes])[1]
    return buffer, byte_arr[num_bytes+1:end]
end

function __read_int(byte_arr, num_bytes = TYPE_INTEGER_LEN)
    buffer = byte_arr[1:num_bytes]
    integer = reinterpret(Int32, buffer)[1]
    return integer, byte_arr[num_bytes+1:end]
end

function __read_string(byte_arr, num_bytes = TYPE_STRING_LEN, dostrip = true)
    buffer = String(byte_arr[1:num_bytes])
    if dostrip
        buffer = strip(buffer, '\x00')
    end
    return buffer, byte_arr[num_bytes+1:end]
end

function parse_header(byte_arr)
    header = Dict()

    header["header"], byte_arr = __read_string(byte_arr, 8, false)

    if pop!(header, "header") != HEADER
        error("Bad file format!")
    end


    header["DEMOPROTOCOL"], byte_arr = __read_int(byte_arr)
    header["NETWORKPROTOCOL"], byte_arr = __read_int(byte_arr)
    header["SERVERNAME"], byte_arr = __read_string(byte_arr)
    header["CLIENTNAME"], byte_arr = __read_string(byte_arr)
    header["MAPNAME"], byte_arr = __read_string(byte_arr)
    header["GAMEDIRECTORY"], byte_arr = __read_string(byte_arr)
    header["PLAYBACKTIME"], byte_arr = __read_float(byte_arr)
    header["TICKS"], byte_arr = __read_int(byte_arr)
    header["FRAMES"], byte_arr = __read_int(byte_arr)
    header["SIGNONLENGTH"], byte_arr = __read_int(byte_arr)
    header["TICKRATE"] = header["TICKS"] / header["PLAYBACKTIME"]

    if isnan(header["TICKRATE"])
        error("demo has nan tickrate, likely 0 tick")
    end
    return header, byte_arr

end

parse_header (generic function with 1 method)

In [6]:
function parseDemo(path::AbstractString)
    ## Parse Portal2 demos based on https://developer.valvesoftware.com/wiki/DEM_Format ##

    buffer = read(path)
    parts = split(path, "_")

    if isempty(buffer)
        error("File Is Empty")
    end

    #### HEADER SHIT ####

    header, byte_array = parse_header(buffer)
    header["STEAMID"] = parse(Int64, split(parts[3], ".")[1])
    header["DEMONAME"] = basename(path)

    return header
end

parseDemo (generic function with 1 method)

In [7]:
function dict2Tuple(value::Dict)
    d = value
    return NamedTuple(Symbol(k) => v for (k, v) in d)
end

dict2Tuple (generic function with 1 method)

In [11]:
function allToDB(path::AbstractString)
    itr = walkdir(path)
    existingFiles = eachcol(
        DataFrames.DataFrame(DBInterface.execute(db, "SELECT DEMONAME FROM HEADERS")),
    )[1]
    stmt = DBInterface.prepare(
        db,
        "INSERT INTO HEADERS (DEMONAME, DEMOPROTOCOL, NETWORKPROTOCOL, SERVERNAME, CLIENTNAME, MAPNAME, GAMEDIRECTORY, PLAYBACKTIME, TICKS, FRAMES, SIGNONLENGTH, TICKRATE, STEAMID) VALUES (:DEMONAME, :DEMOPROTOCOL, :NETWORKPROTOCOL, :SERVERNAME, :CLIENTNAME, :MAPNAME, :GAMEDIRECTORY, :PLAYBACKTIME, :TICKS, :FRAMES, :SIGNONLENGTH, :TICKRATE, :STEAMID)",
    )
    for (root, dirs, files) in itr
        for file in files
            if file ∉ existingFiles
                try
                    DBInterface.execute(stmt, dict2Tuple(parseDemo(joinpath(path, file))))
                    SQLite.commit(db)
                catch
                end
            end
        end
    end
end


allToDB (generic function with 1 method)

In [12]:
allToDB("testingDemos")

In [13]:
allToDB("../p2demos/")